# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 69 new papers today
          11 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/10 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2302.12911


extracting tarball to tmp_2302.12911...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.12970


extracting tarball to tmp_2302.12970...

...

extracting tarball to tmp_2302.13899...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.14023


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2302.13899/mnras_template.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2302.14023...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.12911-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.12911) | **CMB constraints on the early universe independent of late time cosmology**  |
|| Pablo Lemos, <mark>Antony Lewis</mark> |
|*Appeared on*| *2023-02-28*|
|*Comments*| *9 pages, 5 figures*|
|**Abstract**| The CMB is a powerful probe of early-universe physics but is only observed after passing through large-scale structure, which changes the observed spectra in important model-dependent ways. This is of particular concern given recent claims of significant discrepancies with low redshift data sets when a standard $\Lambda$CDM model is assumed. By using empirical measurements of the CMB lensing reconstruction, combined with weak priors on the smoothness of the lensing spectrum, foregrounds, and shape of any additional integrated Sachs-Wolfe effect, we show how the early-universe parameters can be constrained from CMB observations almost independently of the late-time evolution. This provides a way to test new models for early-universe physics, and measure early-universe parameters, independently of late-time cosmology. Using the empirical measurement of lensing keeps the size of the effect of late-time modelling uncertainty under control, leading to only modest increases in error bars of most early-universe parameters compared to assuming a full evolution model. We provide robust constraints on early-$\Lambda$CDM model parameters using the latest Planck PR4 data and show that with future data marginalizing over a single lensing amplitude parameter is sufficient to remove sensitivity to late-time cosmological model only if the spectral shape matches predictions. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.12970-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.12970) | **Abundance ratios of OH/CO and HCO+/CO as probes of the cosmic ray  ionization rate in diffuse clouds**  |
|| Gan Luo, et al. -- incl., <mark>Zhiyu Zhang</mark>, <mark>Di Li</mark> |
|*Appeared on*| *2023-02-28*|
|*Comments*| *22 pages, 9 figures, accepted by ApJ*|
|**Abstract**| The cosmic-ray ionization rate (CRIR, $\zeta_2$) is one of the key parameters controlling the formation and destruction of various molecules in molecular clouds. However, the current most commonly used CRIR tracers, such as H$_3^+$, OH$^+$, and H$_2$O$^+$, are hard to detect and require the presence of background massive stars for absorption measurements. In this work, we propose an alternative method to infer the CRIR in diffuse clouds using the abundance ratios of OH/CO and HCO$^+$/CO. We have analyzed the response of chemical abundances of CO, OH, and HCO$^+$ on various environmental parameters of the interstellar medium in diffuse clouds and found that their abundances are proportional to $\zeta_2$. Our analytic expressions give an excellent calculation of the abundance of OH for $\zeta_2$ $\leq$10$^{-15}$ s$^{-1}$, which are potentially useful for modelling chemistry in hydrodynamical simulations. The abundances of OH and HCO$^+$ were found to monotonically decrease with increasing density, while the CO abundance shows the opposite trend. With high-sensitivity absorption transitions of both CO (1--0) and (2--1) lines from ALMA, we have derived the H$_2$ number densities ($n_{\rm H_2}$) toward 4 line-of-sights (LOSs); assuming a kinetic temperature of $T_{\rm k}=50\,{\rm K}$, we find a range of (0.14$\pm$0.03--1.2$\pm$0.1)$\times$10$^2$ cm$^{-3}$}. By comparing the observed and modelled HCO$^+$/CO ratios, we find that $\zeta_2$ in our diffuse gas sample is in the { range of $1.0_{-1.0}^{+14.8}$ $\times$10$^{-16}- 2.5_{-2.4}^{+1.4}$ $\times$10$^{-15}$ s$^{-1}$. This is $\sim$2 times higher than the average value measured at higher extinction, supporting an attenuation of CRs as suggested by theoretical models. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.13354-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.13354) | **Direct Imaging Explorations for Companions around Mid-Late M Stars from  the Subaru/IRD Strategic Program**  |
|| Taichi Uyama, et al. -- incl., <mark>Akitoshi Ueda</mark> |
|*Appeared on*| *2023-02-28*|
|*Comments*| *16 pages, 8 figures, accepted for publication in AJ*|
|**Abstract**| The Subaru telescope is currently performing a strategic program (SSP) using the high-precision near-infrared (NIR) spectrometer IRD to search for exoplanets around nearby mid/late-M~dwarfs via radial velocity (RV) monitoring. As part of the observing strategy for the exoplanet survey, signatures of massive companions such as RV trends are used to reduce the priority of those stars. However, this RV information remains useful for studying the stellar multiplicity of nearby M~dwarfs. To search for companions around such ``deprioritized" M~dwarfs, we observed 14 IRD-SSP targets using Keck/NIRC2 observations with pyramid wavefront sensing at NIR wavelengths, leading to high sensitivity to substellar-mass companions within a few arcseconds. We detected two new companions (LSPM~J1002+1459~B and LSPM~J2204+1505~B) and two new candidates that are likely companions (LSPM~J0825+6902~B and LSPM~J1645+0444~B) as well as one known companion. Including two known companions resolved by the IRD fiber injection module camera, we detected seven (four new) companions at projected separations between $\sim2-20$~au in total. A comparison of the colors with the spectral library suggests that LSPM~J2204+1505~B and LSPM~J0825+6902~B are located at the boundary between late-M and early-L spectral types. Our deep high-contrast imaging for targets where no bright companions were resolved did not reveal any additional companion candidates. The NIRC2 detection limits could constrain potential substellar-mass companions ($\sim10-75\ M_{\rm Jup}$) at 10~au or further. The failure with Keck/NIRC2 around the IRD-SSP stars having significant RV trends makes these objects promising targets for further RV monitoring or deeper imaging with JWST to search for smaller-mass companions below the NIRC2 detection limits. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.13884-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.13884) | **Jet-induced molecular gas excitation and turbulence in the Teacup**  |
|| A. Audibert, et al. -- incl., <mark>C. Ramos Almeida</mark> |
|*Appeared on*| *2023-02-28*|
|*Comments*| *9 pages, 8 figures. Accepted for publication in A&A Letters*|
|**Abstract**| In order to investigate the impact of radio jets on the interstellar medium (ISM) of galaxies hosting active galactic nuclei (AGN), we present subarcsecond resolution Atacama Large Millimeter/submillimeter Array (ALMA) CO(2-1) and CO(3-2) observations of the Teacup galaxy. This is a nearby ($D_{\rm L}$=388 Mpc) radio-quiet type-2 quasar (QSO2) with a compact radio jet ($P_{\rm jet}\approx$10$^{43}$ erg s$^{-1}$) that subtends a small angle from the molecular gas disc. Enhanced emission line widths perpendicular to the jet orientation have been reported for several nearby AGN for the ionised gas. For the molecular gas in the Teacup, not only do we find this enhancement in the velocity dispersion but also a higher brightness temperature ratio (T32/T21) perpendicular to the radio jet compared to the ratios found in the galaxy disc. Our results and the comparison with simulations suggest that the radio jet is compressing and accelerating the molecular gas, and driving a lateral outflow that shows enhanced velocity dispersion and higher gas excitation. These results provide further evidence that the coupling between the jet and the ISM is relevant to AGN feedback even in the case of radio-quiet galaxies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.13899-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.13899) | **The 2019 outburst of AMXP SAX J1808.4-3658 and radio follow up of MAXI  J0911-655 and XTE J1701-462**  |
|| K.V.S. Gasealahwe, et al. -- incl., <mark>M. Klein-Wolt</mark> |
|*Appeared on*| *2023-02-28*|
|*Comments*| *Contains 9 pages and 3 figures, accepted for publication in MNRAS*|
|**Abstract**| We present radio coverage of the 2019 outburst of the accreting millisecond X-ray pulsar SAX J1808.4-3658, obtained with MeerKAT. We compare these data to contemporaneous X-ray and optical measurements in order to investigate the coupling between accretion and jet formation in this system, while the optical lightcurve provides greater detail of the outburst. The reflaring activity following the main outburst peak was associated with a radio re-brightening, indicating a strengthening of the jet in this phase of the outburst. We place quasi-simultaneous radio and X-ray measurements on the global radio:X-ray plane for X-ray binaries, and show they reside in the same region of luminosity space as previous outburst measurements, but significantly refine the correlation for this source. We also present upper limits on the radio emission from the accreting millisecond X-ray pulsar MAXI J0911-655 and the transitional Z/Atoll-type transient XTE J1701-462. In the latter source we also confirm that nearby large-scale structures reported in previous radio observations of the source are persistent over a period of ~15 years, and so are almost certainly background radio galaxies and not associated with the X-ray transient. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.14023-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.14023) | **New mechanisms for forming multiple hotspots in radio jets**  |
|| Maya A. Horton, <mark>Martin G. H. Krause</mark>, Martin J. Hardcastle |
|*Appeared on*| *2023-02-28*|
|*Comments*| *14 pages, 10 figures*|
|**Abstract**| Hotspots of radio galaxies are regions of shock-driven particle acceleration. Multiple hotspots have long been identified as potential indicators of jet movement or precession. Two frequent explanations describe a secondary hotspot as either the location of a prior jet termination point, or a deflected backflow-driven shock: the so-called Dentist's Drill and Splatter Spot models. We created high-resolution simulations of precessing jets with a range of parameters. In addition to the existing mechanisms, our results show three additional mechanisms for multiple hotspot formation: (1) the splitting of a large terminal hotspots into passive and active components; (2) jet stream splitting resulting in two active hotspots; (3) dynamic multiple hotspot complexes that form as a result of jet termination in a turbulent cocoon, linked here to rapid precession. We show that these distinct types of multiple hotspots are difficult to differentiate in synthetic radio maps, particularly hotspot complexes which can easily be mistaken for the jet itself. We discuss the implication for hypothesised binary supermassive black hole systems where jet precession is a key component of the morphology, and show a selection of potential precession candidates found using the LOFAR Two-Metre Sky Survey Data Release 2 (LoTSS DR2). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.13303-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.13303) | **Temporally distributed parent body accretion in the C reservoir of the  solar system**  |
|| <mark>Wladimir Neumann</mark>, Ning Ma, Audrey Bouvier, Mario Trieloff |
|*Appeared on*| *2023-02-28*|
|*Comments*| **|
|**Abstract**| Accretion processes in protoplanetary disks produce a diversity of small bodies that played a crucial role in multiple reshuffling events throughout the solar system and in both early and late accretion of planets. Application of thermo-chronometers to meteorites provides precise dating of the formation age of mineralogical components. Nucleosynthetic anomalies indicating a dichotomy between NC and C meteorites and precise parent body (PB) chronology can be combined with thermal evolution models to constrain the timescale of accretion and dynamical processes in the early solar system. Achondrite PBs are considered to have accreted early and mostly in the NC region, while late accretion in the C region produced mostly undifferentiated PBs, such as the CR chondrite PB that formed as late as 4 Ma after CAIs. However, presence of more evolved CR-related meteorites suggests also an earlier accretion timing. We present modeling evidence for a temporally distributed accretion of parent bodies of CR-related meteorite groups that originate from a C reservoir and range from aqueously altered chondrites to partially differentiated primitive achondrites. The PB formation times derived range from <1 Ma to ~4 Ma after solar system formation, with ~3.7 Ma, ~1.5-2.75 Ma, <~0.6 Ma, and <~0.7 Ma for CR, Flensburg, NWA 6704, and NWA 011. This implies that accretion processes in the C reservoir started as early as in the NC reservoir and produced differentiated PBs with carbonaceous compositions in addition to undifferentiated C chondritic PBs. The accretion times correlate inversely with the degree of the meteorites' alteration, metamorphism, or differentiation. Accretion times for CI/CM, Ryugu, and Tafassites PBs of ~3.75 Ma, ~1-3 Ma, and 1.1 Ma, respectively, fit well into this correlation in agreement with the thermal and alteration conditions suggested by the meteorites. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.13544-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.13544) | **Systematic KMTNet Planetary Anomaly Search. VIII. Complete Sample of  2019 Subprime Field Planets**  |
|| Youn Kil Jung, et al. -- incl., <mark>Andrew Gould</mark>, <mark>Seung-Lee Kim</mark>, <mark>Chung-Uk Lee</mark>, <mark>Dong-Joo Lee</mark>, <mark>Yongseok Lee</mark>, <mark>Grant Christie</mark>, <mark>Jonathan Green</mark> |
|*Appeared on*| *2023-02-28*|
|*Comments*| *17 tables, 20 figures, submitted to AAS journals*|
|**Abstract**| We complete the publication of all microlensing planets (and ``possible planets'') identified by the uniform approach of the KMT AnomalyFinder system in the 21 KMT subprime fields during the 2019 observing season, namely KMT-2019-BLG-0298, KMT-2019-BLG-1216, KMT-2019-BLG-2783, OGLE-2019-BLG-0249, and OGLE-2019-BLG-0679 (planets), as well as OGLE-2019-BLG-0344, and KMT-2019-BLG-0304 (possible planets). The five planets have mean log mass-ratio measurements of $(-2.6,-3.6,-2.5,-2.2,-2.3)$, median mass estimates of $(1.81,0.094,1.16,7.12,3.34)\, M_{\rm Jup}$, and median distance estimates of $(6.7,2.7,5.9,6.4,5.6)\, {\rm kpc}$, respectively. The main scientific interest of these planets is that they complete the AnomalyFinder sample for 2019, which has a total of 25 planets that are likely to enter the statistical sample. We find statistical consistency with the previously published 33 planets from the 2018 AnomalyFinder analysis according to an ensemble of five tests. Of the 58 planets from 2018-2019, 23 were newly discovered by AnomalyFinder. Within statistical precision, half of all the planets have caustic crossings while half do not (as predicted by Zhu et al. 2014), an equal number of detected planets result from major-image and minor-image light-curve perturbations, and an equal number come from KMT prime fields versus subprime fields. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error [Errno 2] No such file or directory: 'gs'</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

214  publications in the last 7 days.
	 _build/html/2302.12805.md
	 _build/html/2302.10943.md
	 _build/html/2302.10528.md
	 _build/html/2302.10008.md
	 _build/html/2302.08962.md
	 _build/html/2302.08628.md
	 _build/html/2302.07916.md
	 _build/html/2302.07880.md
	 _build/html/2302.07497.md
	 _build/html/2302.07277.md
	 _build/html/2302.07256.md
	 _build/html/2302.07234.md
	 _build/html/2302.07057.md
	 _build/html/2302.05694.md
	 _build/html/2302.05465.md
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.03039.md
	 _build/html/2302.02890.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

3  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> daily </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.
